#### Redo simlex999 word pair similarity rating exercise with w2v models, e.g. wiki.en.vec and/or your own home-grown model (via gensim or fasttext)

Web-public corpus (1M) from https://wortschatz.uni-leipzig.de/en/download/English

In [1]:
with open('eng-com_web-public_2018_1M-sentences.txt', 'r') as file:
    corpus = file.read().lower().replace('\t', '')

In [2]:
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
import re

In [3]:
corpus_sent = sent_tokenize(corpus)

In [4]:
def decontracted(sentence):
    # specific
    phrase = re.sub(r"won\'t", "will not", sentence)
    phrase = re.sub(r"can\'t", "can not", sentence)

    # general
    phrase = re.sub(r"n\'t", " not", sentence)
    phrase = re.sub(r"\'re", " are", sentence)
    phrase = re.sub(r"\'s", " is", sentence)
    phrase = re.sub(r"\'d", " would", sentence)
    phrase = re.sub(r"\'ll", " will", sentence)
    phrase = re.sub(r"\'t", " not", sentence)
    phrase = re.sub(r"\'ve", " have", sentence)
    phrase = re.sub(r"\'m", " am", sentence)
    return phrase

extended_corpus = [decontracted(sentence) for sentence in corpus_sent]

In [5]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r"[?|$|&|*|%|@|(|)|~]"
        filtered_sentence = re.sub(PATTERN, r"", sentence)
    else:
        PATTERN = r"[^a-zA-Z ]"
        filtered_sentence = re.sub(PATTERN, r"", sentence)
    return filtered_sentence.strip()

cleaned_corpus = [remove_characters_before_tokenization(sentence) for sentence in extended_corpus]

In [9]:
stopword_list = nltk.corpus.stopwords.words('english')
def remove_stopwords(sentence):
    tokens = word_tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

corpus_w_stopwords = [remove_stopwords(sentence) for sentence in cleaned_corpus]

In [11]:
corpus_final = [word_tokenize(sentence) for sentence in corpus_w_stopwords]

https://fasttext.cc/docs/en/crawl-vectors.html ... trained on Common Crawl and Wikipedia using fastText. These models were trained using CBOW with position-weights, in dimension 300, with character n-grams of length 5, a window of size 5 and 10 negatives.

In [12]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format("cc.en.300.vec") # wiki.en.vec
model_2 = gensim.models.Word2Vec(corpus_final, size=300, window=10, min_count=10, sample=1e-3) # my model

In [13]:
import pandas as pd
simlex999 = pd.read_csv("SimLex-999.txt", sep="\t")
simlex999

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [14]:
import numpy as np
from scipy import stats

simlex_scores = []
w2v_wiki_scores = []
my_model_scores = []
for x, y, simlex_score in zip(simlex999.word1, simlex999.word2, simlex999.SimLex999):
    try:
        wiki_score = model.wv.similarity(x, y)
        my_model_score = model_2.wv.similarity(x, y)
        if my_model_score == "None":
            raise ValueError("None")
            
        simlex_scores.append(simlex_score)
        w2v_wiki_scores.append(wiki_score)
        my_model_scores.append(my_model_score)
        
    except:
        continue
    
print("w2v wiki score: ",stats.pearsonr(w2v_wiki_scores,simlex_scores))
print("my model score: ",stats.pearsonr(my_model_scores,simlex_scores))

w2v wiki score:  (0.48450447967244925, 1.818804540998229e-59)
my model score:  (0.2923378271374335, 5.599965145998254e-21)


<ipython-input-14-1bcb3af33b4c>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  wiki_score = model.wv.similarity(x, y)
